In [21]:
#environment setup
import pandas as pd
import requests
import os
from pathlib import Path
from dotenv import load_dotenv

# Load .env from the project root (parent of data_cleaning)
load_dotenv(Path.cwd().resolve().parent / ".env")
GOOGLE_SHEETS_API_KEY = os.getenv('GOOGLE_SHEETS_API_KEY')
PROJECTS_SHEET_ID = os.getenv('PROJECTS_SHEET_ID')
ACTIVES_SHEET_ID = os.getenv('ACTIVES_SHEET_ID')
FALL_ATTENDANCE_SHEET_ID = os.getenv('FALL_ATTENDANCE_SHEET_ID')
BASE_URL = "https://sheets.googleapis.com/v4/spreadsheets"

In [22]:
def get_sheet_titles(spreadsheet_id: str, api_key: str) -> list[str]:
    """Return all sheet/tab titles in the spreadsheet."""
    resp = requests.get(
        f"{BASE_URL}/{spreadsheet_id}",
        params={
            "fields": "sheets(properties(title))",
            "key": api_key,
        },
        timeout=30,
    )
    resp.raise_for_status()
    data = resp.json()
    return [s["properties"]["title"] for s in data.get("sheets", [])]

In [23]:
def fetch_values_batch(spreadsheet_id: str, api_key: str, sheet_titles: list[str]) -> dict[str, list[list]]:
    """Batch fetch values for provided sheet titles.
    Returns mapping of title -> 2D list of cell values (including header row).
    """
    if not sheet_titles:
        return {}
    # Multiple 'ranges' params are supported by the API
    params = [("key", api_key), ("valueRenderOption", "UNFORMATTED_VALUE"), ("dateTimeRenderOption", "FORMATTED_STRING")]
    params.extend(("ranges", title) for title in sheet_titles)
    resp = requests.get(
        f"{BASE_URL}/{spreadsheet_id}/values:batchGet",
        params=params,
        timeout=60,
    )
    resp.raise_for_status()
    payload = resp.json()

    values_by_title: dict[str, list[list]] = {}
    for vr, title in zip(payload.get("valueRanges", []), sheet_titles):
        values_by_title[title] = vr.get("values", [])
    return values_by_title

In [24]:
def values_to_dataframe(values: list[list]) -> pd.DataFrame:
    """Convert a 2D list from Sheets API to a DataFrame, using first row as header.
    Pads short rows so all rows match header length.
    """
    if not values:
        return pd.DataFrame()
    header = [str(h) for h in values[0]]
    rows = values[1:] if len(values) > 1 else []
    normalized_rows = [row + [""] * (len(header) - len(row)) for row in rows]
    return pd.DataFrame(normalized_rows, columns=header)

In [25]:
def fetch_spreadsheet_as_dataframes(spreadsheet_id: str, api_key: str) -> dict[str, pd.DataFrame]:
    """Fetch all sheets in a spreadsheet and return {sheet_title: DataFrame}."""
    titles = get_sheet_titles(spreadsheet_id, api_key)
    if not titles:
        return {}
    values_by_title = fetch_values_batch(spreadsheet_id, api_key, titles)
    return {title: values_to_dataframe(values_by_title.get(title, [])) for title in titles}

In [26]:
# fetch all data from Google Sheets into DataFrames per sheet
# Build DataFrames per sheet for each spreadsheet ID found in .env
PROJECTS_DFS = fetch_spreadsheet_as_dataframes(PROJECTS_SHEET_ID, GOOGLE_SHEETS_API_KEY) if PROJECTS_SHEET_ID else {}
ACTIVES_DFS = fetch_spreadsheet_as_dataframes(ACTIVES_SHEET_ID, GOOGLE_SHEETS_API_KEY) if ACTIVES_SHEET_ID else {}
FALL_ATTENDANCE_DFS = fetch_spreadsheet_as_dataframes(FALL_ATTENDANCE_SHEET_ID, GOOGLE_SHEETS_API_KEY) if FALL_ATTENDANCE_SHEET_ID else {}


HTTPError: 403 Client Error: Forbidden for url: https://sheets.googleapis.com/v4/spreadsheets/1PVSfpJyuFB7uyVbSVZgp_G0H_upWQUmnmOB4C15NxK0?fields=sheets%28properties%28title%29%29